# Lesson 09 - Workalong 2, Step 1
This notebook develops and tests the code for a function that generates a plot of some interesting weather.

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cartopy
import cmocean
import os

In [ ]:
# set the path to the input data file
input_file = "/N/project/obrienta_startup/datasets/ERA5/ds633.0/e5.oper.an.sfc/202106/e5.oper.an.sfc.128_136_tcw.ll025sc.2021060100_2021063023.nc"

# set the directory where we'll save the images

# set the timestep to plot

# make sure the output directory exists

# load the data file

In [ ]:
# get the variable at the requested timestep

# plot the variable on an orthographic projection centered on
# Bloomington, IN


# plot the data


# get the time of the timestep

# convert it to a datetime object

# add a title with a nicely formatted date

# add coastlines

# save the plot

In [ ]:
""" Make a function to plot and save the data at a given timestep. """

def generate_frame(
        i : int,
        input_file = "/N/project/obrienta_startup/datasets/ERA5/ds633.0/e5.oper.an.sfc/202106/e5.oper.an.sfc.128_136_tcw.ll025sc.2021060100_2021063023.nc",
        output_dir = "./animation_frames/"):

        # copy the code from above here; have this function return the output file name
        pass



In [ ]:
""" Test the function. """

# set the timestep to plot (select a new timestep)

# generate the plot